## Data Visualization 

In [1]:
import pandas as pd
import numpy as np
import folium
from collections import defaultdict
import branca.colormap
from folium import plugins
from flood_tool import plot_circle
import haversine as hs
from ipywidgets import interact

In [2]:
wet_rain = pd.read_csv('wet_rainfall.csv')[['dateTime', 'stationReference','wet_day', 'latitude', 'longitude', 'aclass']]
wet_river = pd.read_csv('wet_river.csv')[['dateTime', 'stationReference','wet_day', 'latitude', 'longitude', 'aclass']]
typ_rain = pd.read_csv('typ_rainfall.csv')[['dateTime', 'stationReference','typical_day', 'latitude', 'longitude']]
typ_river = pd.read_csv('typ_river.csv')[['dateTime', 'stationReference','typical_day', 'latitude', 'longitude']]

### Data processing 
Taking the mean rainfall and river data for each station over the day and take log10 for visualization.

In [3]:
wet_rain = pd.read_csv('wet_rainfall.csv')[['dateTime', 'stationReference','wet_day', 'latitude', 'longitude', 'aclass']]
wet_rain_df = wet_rain.groupby('stationReference') \
       .agg({'stationReference':'size', 'wet_day':'mean'}) \
       .rename(columns={'stationReference':'count','wet_day':'mean_rainfall'}) \
       .reset_index()

wet_rain_merge = pd.merge(wet_rain_df, wet_rain[["stationReference",'latitude','longitude']], on="stationReference")
wet_rain_mean = wet_rain_merge.drop_duplicates()
wet_rain_mean[['wet_day_log']] = np.log10(wet_rain_mean[['mean_rainfall']])
wet_rain_mean = wet_rain_mean.reset_index(drop=True)

In [4]:
wet_river_df = wet_river.groupby('stationReference') \
       .agg({'stationReference':'size', 'wet_day':'mean'}) \
       .rename(columns={'stationReference':'count','wet_day':'mean_river'}) \
       .reset_index()

wet_river_merge = pd.merge(wet_river_df, wet_river[["stationReference",'latitude','longitude']], on="stationReference")
wet_river_mean = wet_river_merge.drop_duplicates()
wet_river_mean[['wet_day_log']] = np.log10(wet_river_mean[['mean_river']], where=wet_river_mean[['mean_river']]>0)
wet_river_mean = wet_river_mean.reset_index(drop=True)

In [5]:
typ_rain_df = typ_rain.groupby('stationReference') \
       .agg({'stationReference':'size', 'typical_day':'mean'}) \
       .rename(columns={'stationReference':'count','typical_day':'mean_rainfall'}) \
       .reset_index()

typ_rain_merge = pd.merge(typ_rain_df, typ_rain[["stationReference",'latitude','longitude']], on="stationReference")
typ_rain_mean = typ_rain_merge.drop_duplicates()
typ_rain_mean[['typ_day_log']] = np.log10(typ_rain_mean[['mean_rainfall']])
typ_rain_mean = typ_rain_mean.reset_index(drop=True)

In [6]:
typ_river_df = typ_river.groupby('stationReference') \
       .agg({'stationReference':'size', 'typical_day':'mean'}) \
       .rename(columns={'stationReference':'count','typical_day':'mean_river'}) \
       .reset_index()

typ_river_merge = pd.merge(typ_river_df, typ_river[["stationReference",'latitude','longitude']], on="stationReference")
typ_river_mean = typ_river_merge.drop_duplicates()
typ_river_mean[['typ_day_log']] = np.log10(typ_river_mean[['mean_river']], where=typ_river_mean[['mean_river']]>0)
typ_river_mean = typ_river_mean.reset_index(drop=True)

For rainfall data, a measurement equals to zero means no rain, so after taking log those values become negative infinity, which shows up nicely in the plot. However, for river data, zero river level means a very low river level, and we keep the zeroes after taking the log. 

### Visualization of mean 15-min rainfall in a wet day

In [7]:
colormap = branca.colormap.LinearColormap(colors=['white','blue'], index=[-5,0.3],vmin=-5,vmax=0.3)
map = folium.Map(location=[52.4862, -0.118092], zoom_start=6)
colormap.add_to(map)

folium.TileLayer('Stamen Terrain').add_to(map)
folium.LayerControl().add_to(map)

for loc, p in zip(zip(wet_rain_mean["latitude"], wet_rain_mean["longitude"]), wet_rain_mean["wet_day_log"]):
    folium.Circle(
        location=loc,
        radius=10,
        fill=True,
        color=colormap(p),
        fill_opacity=0.7
    ).add_to(map)

map.add_child(colormap)
map

### Visualization of mean 15-min rainfall in a typical day

In [8]:
colormap = branca.colormap.LinearColormap(colors=['white','blue'], index=[-5,0.3],vmin=-5,vmax=0.3)
map = folium.Map(location=[52.4862, -0.118092], zoom_start=6)
colormap.add_to(map)

folium.TileLayer('Stamen Terrain').add_to(map)
folium.LayerControl().add_to(map)

for loc, p in zip(zip(typ_rain_mean["latitude"], typ_rain_mean["longitude"]), typ_rain_mean["typ_day_log"]):
    folium.Circle(
        location=loc,
        radius=10,
        fill=True,
        color=colormap(p),
        fill_opacity=0.7
    ).add_to(map)

map.add_child(colormap)
map

### Visualization of mean river level in a wet day

In [9]:
colormap = branca.colormap.LinearColormap(colors=['yellow','red'], 
                                          index=[-2,1],vmin=-3,vmax=1.85)
map = folium.Map(location=[52.4862, -0.118092], zoom_start=6)
colormap.add_to(map)

folium.TileLayer('Stamen Terrain').add_to(map)
folium.LayerControl().add_to(map)

for loc, p in zip(zip(wet_river_mean["latitude"], wet_river_mean["longitude"]), wet_river_mean["wet_day_log"]):
    folium.Circle(
        location=loc,
        radius=10,
        fill=True,
        color=colormap(p),
        fill_opacity=0.7
    ).add_to(map)

map.add_child(colormap)
map

### Visualization of mean river level in a wet day

In [10]:
colormap = branca.colormap.LinearColormap(colors=['yellow','red'], 
                                          index=[-2,1],vmin=-3,vmax=1.85)
map = folium.Map(location=[52.4862, -0.118092], zoom_start=6)
colormap.add_to(map)

folium.TileLayer('Stamen Terrain').add_to(map)
folium.LayerControl().add_to(map)

for loc, p in zip(zip(typ_river_mean["latitude"], typ_river_mean["longitude"]), typ_river_mean["typ_day_log"]):
    folium.Circle(
        location=loc,
        radius=10,
        fill=True,
        color=colormap(p),
        fill_opacity=0.7
    ).add_to(map)

map.add_child(colormap)
map

### Interactive map for rainfall data in a wet day with changing time

In [11]:
@interact
def pick_time(SelectTime=wet_rain['dateTime'].unique()):
    wet_rain_time = wet_rain[wet_rain['dateTime'] == SelectTime]
    wet_rain_df = wet_rain_time.groupby('stationReference') \
       .agg({'stationReference':'size', 'wet_day':'mean'}) \
       .rename(columns={'stationReference':'count','wet_day':'mean_rainfall'}) \
       .reset_index()

    wet_rain_merge = pd.merge(wet_rain_df, wet_rain[["stationReference",'latitude','longitude']], on="stationReference")
    wet_rain_mean = wet_rain_merge.drop_duplicates()
    wet_rain_mean[['wet_day_log']] = np.log10(wet_rain_mean[['mean_rainfall']])
    
    i_max = wet_rain_mean['wet_day_log'].max()
    i_min = wet_rain_mean['wet_day_log'].min()
    
    colormap = branca.colormap.LinearColormap(colors=['white','blue'], index=[-5,1],vmin=-5,vmax=1)
    map = folium.Map(location=[52.4862, -0.118092], zoom_start=6)
    colormap.add_to(map)

    folium.TileLayer('Stamen Terrain').add_to(map)
    folium.LayerControl().add_to(map)

    for loc, p in zip(zip(wet_rain_mean["latitude"], wet_rain_mean["longitude"]), wet_rain_mean["wet_day_log"]):
        folium.Circle(
            location=loc,
            radius=10,
            fill=True,
            color=colormap(p),
            fill_opacity=0.7
        ).add_to(map)

    map.add_child(colormap)

    return map

interactive(children=(Dropdown(description='SelectTime', options=('2021-05-07T00:00:00Z', '2021-05-07T00:15:00…

### Using functions in tool.py to plot added information on map

In [12]:
import sys
sys.path.append('./flood_tool')
from geo import *
from tool import *
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [13]:
df1 = pd.read_csv('./flood_tool/resources/postcodes_unlabelled.csv')
df1.head()
tool = Tool()

### Calculate regional average rainfall and river data given any postcode location
Here we calculate the average of all rainfall or river daily means within 50 km of a given location (location needs to be in the unlabelled data).  

In [14]:
def get_wet_rain(loc):
    wet_rain_df = wet_rain.groupby('stationReference') \
       .agg({'stationReference':'size', 'wet_day':'mean'}) \
       .rename(columns={'stationReference':'count','wet_day':'mean_rainfall'}) \
       .reset_index()

    wet_rain_merge = pd.merge(wet_rain_df, wet_rain[["stationReference",'latitude','longitude']], on="stationReference")
    wet_rain_mean = wet_rain_merge.drop_duplicates()
    wet_rain_mean = wet_rain_mean.reset_index(drop=True)
    dis = []
    for i in range(len(wet_rain_mean['latitude'])):
        lat = wet_rain_mean['latitude'][i]
        long = wet_rain_mean['longitude'][i]
        loc1 = (lat, long)
        dis.append(hs.haversine(loc,loc1))
        
    wet_rain_mean[['distance']] = pd.DataFrame(dis)
    regional_mean = wet_rain_mean[wet_rain_mean['distance'] < 50]['mean_rainfall'].mean()
    return regional_mean

In [15]:
def get_wet_river(loc):
    wet_river_df = wet_river.groupby('stationReference') \
       .agg({'stationReference':'size', 'wet_day':'mean'}) \
       .rename(columns={'stationReference':'count','wet_day':'mean_river'}) \
       .reset_index()

    wet_river_merge = pd.merge(wet_river_df, wet_river[["stationReference",'latitude','longitude']], on="stationReference")
    wet_river_mean = wet_river_merge.drop_duplicates()
    wet_river_mean = wet_river_mean.reset_index(drop=True)
    dis = []
    for i in range(len(wet_river_mean['latitude'])):
        lat = wet_river_mean['latitude'][i]
        long = wet_river_mean['longitude'][i]
        loc1 = (lat, long)
        dis.append(hs.haversine(loc,loc1))
        
    wet_river_mean[['distance']] = pd.DataFrame(dis)
    regional_mean = wet_river_mean[wet_river_mean['distance'] < 50]['mean_river'].mean()
    return regional_mean

### Plot information of predicted flood risk class, housing price on rainfall wet day map

In [16]:
def flood_risk_map(postcode):
    colormap = branca.colormap.LinearColormap(colors=['white','blue'], index=[-5,0.3],vmin=-5,vmax=0.3)
    map = folium.Map(location=[52.4862, -0.118092], zoom_start=6)
    colormap.add_to(map)

    folium.TileLayer('Stamen Terrain').add_to(map)
    folium.LayerControl().add_to(map)

    for loc, p in zip(zip(wet_rain_mean["latitude"], wet_rain_mean["longitude"]), wet_rain_mean["wet_day_log"]):
        folium.Circle(
            location=loc,
            radius=10,
            fill=True,
            color=colormap(p),
            fill_opacity=0.7
        ).add_to(map)

    map.add_child(colormap)

    position = tool.get_lat_long([postcode])
    flood_risk_series = tool.get_flood_class([postcode], 1)
    flood_value = flood_risk_series.values[0]
    house_price_series = tool.get_median_house_price_estimate([postcode], 3)
    price_value = round(house_price_series.values[0],2)
    annual_risk = tool.get_annual_flood_risk([postcode])
    risk = round(annual_risk.values[0],1)
    
    loc = (position['lat'], position['lon'])
    reg_rain = round(get_wet_rain(loc),3)
    reg_riv = round(get_wet_river(loc),3)
    
    pop_up = folium.Popup(f'''Postcode: {postcode} <br> 
                          Predicted Flood Risk Class: {flood_value} <br> 
                          Predicted House Price (£): {price_value} <br> 
                          Predicted Annual Flood Risk: {risk} <br>
                          Daily 15-min Rainfall Average (mm): {reg_rain} <br>
                          Daily River Level Average (mASD): {reg_riv}''',
                     max_width=400)
    
    folium.Marker(
      location=[position['lat'], position['lon']],
      popup=pop_up,
    ).add_to(map)
    return map

@interact
def map_all(Postcode=df1.postcode.tolist()):
    return flood_risk_map(Postcode)


interactive(children=(Dropdown(description='Postcode', options=('YO62 4LS', 'DE2 3DA', 'LN5 7RW', 'CM3 1DW', '…